In [ ]:
!nvidia-smi

Fri Mar  3 14:26:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !git clone --branch u7 https://github.com/WongKinYiu/yolov7.git

path = ''
!unzip '{path}'yolov7_seg.zip

Archive:  /content/drive/Shareddrives/P&I/Data & ML/PV&rooftop_project/yolov7_seg.zip
   creating: yolov7_seg/
   creating: yolov7_seg/det/
  inflating: yolov7_seg/.DS_Store    
  inflating: __MACOSX/yolov7_seg/._.DS_Store  
   creating: yolov7_seg/cls/
  inflating: yolov7_seg/README.md    
   creating: yolov7_seg/seg/
   creating: yolov7_seg/.git/
   creating: yolov7_seg/det/classify/
  inflating: yolov7_seg/det/LICENSE  
  inflating: yolov7_seg/det/requirements.txt  
  inflating: yolov7_seg/det/val.py   
   creating: yolov7_seg/det/utils/
   creating: yolov7_seg/det/models/
  inflating: yolov7_seg/det/export.py  
  inflating: yolov7_seg/det/README.md  
  inflating: yolov7_seg/det/train.py  
  inflating: yolov7_seg/det/setup.cfg  
  inflating: yolov7_seg/det/hubconf.py  
   creating: yolov7_seg/det/data/
  inflating: yolov7_seg/det/detect.py  
  inflating: yolov7_seg/cls/classifier.py  
  inflating: yolov7_seg/cls/LICENSE  
  inflating: yolov7_seg/cls/requirements.txt  
  inflating: y

In [ ]:
%cd yolov7_seg/seg

In [ ]:
!pip install -r requirements.txt

In [ ]:
### based on: https://blog.paperspace.com/train-yolov7-custom-data/
import torch
from IPython.display import Image  # for displaying images
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [ ]:
!unzip "/content/drive/Shareddrives/P&I/Data & ML/PV&rooftop_project/yolo_segmentation_data.zip" -d "/content/"

In [ ]:
# create folders for the data after splitting
!mkdir split_data
!mkdir split_data/images
!mkdir split_data/labels
!mkdir split_data/images/train
!mkdir split_data/labels/train
!mkdir split_data/images/val
!mkdir split_data/labels/val
!mkdir split_data/images/test
!mkdir split_data/labels/test

In [ ]:
ims_path = '/content/yolo_data/images/'
annot_path = '/content/yolo_data/labels/'
images = [os.path.join(ims_path, x) for x in os.listdir(ims_path)]
annotations = [os.path.join(annot_path, x) for x in os.listdir(annot_path) if x[-3:] == "txt"]

In [ ]:
images.sort()
annotations.sort()

train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_images, 'split_data/images/train')
move_files_to_folder(val_images, 'split_data/images/val/')
move_files_to_folder(test_images, 'split_data/images/test/')
move_files_to_folder(train_annotations, 'split_data/labels/train/')
move_files_to_folder(val_annotations, 'split_data/labels/val/')
move_files_to_folder(test_annotations, 'split_data/labels/test/')

### training

In [ ]:
import multiprocessing
workers = multiprocessing.cpu_count()
print("number of workers: ", workers)

In [ ]:
!python segment/train.py --data data.yaml --batch 24 --weights '' --cfg yolov7-seg.yaml --epochs 1200 --name yolov7-seg --img 640 --hyp hyp.extended.yaml

segment/train: weights=, cfg=yolov7-seg.yaml, data=data.yaml, hyp=hyp.extended.yaml, epochs=1200, batch_size=24, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train-seg, name=yolov7-seg, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, mask_ratio=4, no_overlap=False
YOLOv5 🚀 2023-2-8 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.025, hsv_s=0.8, hsv_v=0.5, degrees=10.0, translate=0.2, scale=0.9, shear=0.3, perspective=0.0, flipud=0.2, fliplr=0.65, m

##### test

In [ ]:
!python segment/predict.py --weights /content/yolov7_seg/seg/runs/train-seg/yolov7-seg/weights/best.pt --data data.yaml --name yolo_segmentation --source split_data/images/val/ --line-thickness 1 --save-txt --save-masks #--nosave-with-bbox

segment/predict: weights=['/content/yolov7_seg/seg/runs/train-seg/yolov7-seg/weights/best.pt'], source=split_data/images/val/, data=data.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, save_masks=True, nosave_with_bbox=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/predict-seg, name=yolo_segmentation, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2023-2-8 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
yolov7-seg summary: 325 layers, 37874840 parameters, 0 gradients, 142.0 GFLOPs
image 1/100 /content/yolov7_seg/seg/split_data/images/val/10.png: 640x640 2 PVs, 1 Roof type, 1 Roof windows, 16.0ms
image 2/100 /content/yolov7_seg/seg/split_data/images/val/105.png: 640x640 4 PVs, 6 Roof types, 5 Roof windowss, 1 Dormers, 18.4ms
ima

In [ ]:
!python export.py --weights '/content/yolov7_seg/seg/runs/train-seg/yolov7-seg/weights/best.pt' --include engine onnx --data 'data.yaml' --device 0